In [1]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM, TimeDistributed

Using TensorFlow backend.


In [2]:
timesteps=30

In [3]:
def process_data_seq(n_samples,data,n_frames=5,rows=16,cols=16,channels=1):
    x=[]
    y=[]
    for i in range(n_samples - n_frames):
        row_x=[]
        row_y=[]
        for j in range(n_frames):
            row_x.append(list(data[i+j]))
            row_y.append(list(data[i+j+1]))
        #y.append(data[i+n_frames]) 
        row_x=np.array(row_x)
        row_x=row_x.flatten()
        x.append(row_x)
        row_y=np.array(row_y)
        row_y=row_y.flatten()
        y.append(row_y)
    x=np.array(x)
    y=np.array(y)
    
    x=np.reshape(x,(n_samples- n_frames,n_frames,rows*cols*channels))
    y=np.reshape(y,(n_samples- n_frames,n_frames,rows*cols*channels))
    
    return x,y
    

In [22]:
train=np.loadtxt('compressed/x_train.csv',delimiter='\t',dtype='float32')
test=np.loadtxt('compressed/x_test.csv',delimiter='\t',dtype='float32')

In [ ]:
train.shape

In [23]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
scaler.fit(train)
train=scaler.transform(train)
test=scaler.transform(test)

In [40]:
type(scaler.inverse_transform(train))

numpy.ndarray

In [24]:
x_train,y_train=process_data_seq(n_samples=9999,data=train,n_frames=timesteps)
x_test,y_test=process_data_seq(n_samples=999,data=train,n_frames=timesteps)

In [16]:
del(train)
del(test)

In [25]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(9969, 30, 16, 16, 1)
(9969, 30, 16, 16, 1)
(969, 30, 16, 16, 1)
(969, 30, 16, 16, 1)


In [26]:
#x_train=x_train.reshape(x_train.shape[0],timesteps,256)
#y_train=y_train.reshape(y_train.shape[0],timesteps,256)
#x_test=x_test.reshape(x_test.shape[0],timesteps,256)
#y_test=y_test.reshape(y_test.shape[0],timesteps,256)

In [27]:
inputs = Input(name='inputs',shape=(timesteps,256))
layer = LSTM(256, return_sequences=True)(inputs)
layer = LSTM(256,return_sequences=True)(layer)
layer = LSTM(256,return_sequences=True)(layer)
layer = TimeDistributed(Dense(256))(layer)
model = Model(inputs=inputs,outputs=layer)

In [28]:
model.summary()
model.compile(loss='mean_squared_error',optimizer='sgd')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 30, 256)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 256)           525312    
_________________________________________________________________
lstm_5 (LSTM)                (None, 30, 256)           525312    
_________________________________________________________________
lstm_6 (LSTM)                (None, 30, 256)           525312    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 256)           65792     
Total params: 1,641,728
Trainable params: 1,641,728
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.fit(x_train, y_train,
          batch_size=128, epochs=20, shuffle=False,
          validation_data=(x_test, y_test))

Train on 9969 samples, validate on 969 samples
Epoch 1/20
9969/9969 [==============================] - 156s 16ms/step - loss: 0.0611 - val_loss: 0.0663
Epoch 2/20
9969/9969 [==============================] - 152s 15ms/step - loss: 0.0594 - val_loss: 0.0648
Epoch 3/20
9969/9969 [==============================] - 146s 15ms/step - loss: 0.0580 - val_loss: 0.0635
Epoch 4/20
9969/9969 [==============================] - 143s 14ms/step - loss: 0.0568 - val_loss: 0.0624
Epoch 5/20
9969/9969 [==============================] - 144s 14ms/step - loss: 0.0557 - val_loss: 0.0614
Epoch 6/20
9969/9969 [==============================] - 179s 18ms/step - loss: 0.0548 - val_loss: 0.0606
Epoch 7/20
9969/9969 [==============================] - 145s 15ms/step - loss: 0.0540 - val_loss: 0.0599
Epoch 8/20
9969/9969 [==============================] - 145s 15ms/step - loss: 0.0533 - val_loss: 0.0593
Epoch 9/20
9969/9969 [==============================] - 157s 16ms/step - loss: 0.0528 - val_loss: 0.0588
Epoch 10

In [47]:
model.save('lstm_model_deep_normalised.h5')

In [9]:
from keras.models import load_model
model = load_model('lstm_model_deep_normalised.h5')

In [31]:
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import load_model
from keras.models import Sequential

In [32]:
autoencoder = load_model('autoencoder_model.h5')
decoder_model = Sequential()
for layer in autoencoder.layers[9:17]:
    decoder_model.add(layer)

In [45]:
x=x_train[50]

for i in range(1000):
    x=x.reshape(1,timesteps,256)
    y=model.predict(x)
    y_last=y.reshape(timesteps,256)[-1]
    y_last=y_last.reshape(1,256)
    y_last=scaler.inverse_transform(y_last)
    y_decoded = decoder_model.predict(y_last.reshape(1,4,4,16))
    y_decoded = y_decoded.reshape(1,64,64,1)
    plt.gray()
    plt.imsave('output_frames/frame'+str(i)+'.jpg',y_decoded.reshape(64,64))
    x=x.reshape(timesteps,256)
    y=y.reshape(timesteps,256)
    x=x[1:]
    x=np.append(x,y[timesteps-1].reshape(1,256),axis=0)
    

<Figure size 432x288 with 0 Axes>

In [46]:
import os
os.system("ffmpeg -r 15 -i output_frames/frame%01d.jpg -vcodec mpeg4 -y output_video/dance_video.mp4")

0